In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
import os

def prompt_to_close_excel_files():
    messagebox.showinfo("Close Excel Files", "Please ensure all Excel files are closed before proceeding.")

def upload_file(title, filetypes):
    root = tk.Tk()
    root.withdraw() 
    root.attributes('-topmost', True)  
    filepath = filedialog.askopenfilename(title=title, filetypes=filetypes)
    root.destroy()
    if filepath:
        messagebox.showinfo("File Uploaded", f"{title} uploaded successfully.")
    else:
        messagebox.showerror("File Upload Error", "No file was selected.")
    return filepath

def process_files(file_a_path, file_pre_event_path, file_post_event_path):
   
    df_a = pd.read_excel(file_a_path, sheet_name='Template-New Participants', header=3,
                         converters={'Personnel Number': str})
    df_pre_event = pd.read_excel(file_pre_event_path, converters={'PersNo': str})
    df_post_event = pd.read_excel(file_post_event_path, converters={'PersNo': str})

    
    print("Sample Personnel Numbers in File A:", df_a['Personnel Number'].unique()[:5])
    print("Sample PersNo in Pre-event file:", df_pre_event['PersNo'].unique()[:5])
    print("Sample PersNo in Post-event file:", df_post_event['PersNo'].unique()[:5])

    filtered_pre_event = df_pre_event[df_pre_event['PersNo'].isin(df_a['Personnel Number'])]
    filtered_post_event = df_post_event[df_post_event['PersNo'].isin(df_a['Personnel Number'])]

    print("Filtered Pre-event DataFrame shape:", filtered_pre_event.shape)
    print("Filtered Post-event DataFrame shape:", filtered_post_event.shape)

    merged_df = pd.merge(filtered_pre_event[['PersNo', 'BeforeTax_Rate', 'RothActualRt']],
                         filtered_post_event[['PersNo', 'BeforeTax_Rate', 'RothActualRt']],
                         on='PersNo', how='outer', suffixes=('_PreEvent', '_PostEvent'))

    if not merged_df.empty:
        output_path = os.path.join(os.path.dirname(file_a_path), "combined_data.xlsx")
        merged_df.to_excel(output_path, index=False, engine='openpyxl')
        messagebox.showinfo("Process Complete", f"Filtered and combined data has been saved to {output_path}")
    else:
        messagebox.showinfo("No Data", "No matching data found for the selected criteria.")

def main():
    prompt_to_close_excel_files()
    file_a_path = upload_file("Select the Personnel Numbers file (File A)", [("Excel files", "*.xlsx"), ("Excel 97-2003 files", "*.xls")])
    if file_a_path:
        file_pre_event_path = upload_file("Select the Pre Event Contribution file (Alight File 1)", [("Excel files", "*.xlsx")])
        if file_pre_event_path:
            file_post_event_path = upload_file("Select the Post Event Contribution file (Alight File 2)", [("Excel files", "*.xlsx")])
            if file_post_event_path:
                process_files(file_a_path, file_pre_event_path, file_post_event_path)
            else:
                messagebox.showerror("Error", "Post Event Contribution file was not selected.")
        else:
            messagebox.showerror("Error", "Pre Event Contribution file was not selected.")
    else:
        messagebox.showerror("Error", "Personnel Numbers file (File A) was not selected.")

if __name__ == "__main__":
    main()